In [1]:
import numpy as np
from spike_tools import data_preprocessing, estimate_decay_rates, estimate_spikes
from matplotlib import pyplot as plt
%matplotlib notebook

ModuleNotFoundError: No module named 'spike_tools'

## Example 1: An in-vitro data setting

This example illustrates the full pipeline: 
- Load data and preprocess
- Estimate decay rate based on ephys
- Estimate spikes based on estimated decay rate and a target average firing rate

In [ ]:
# Data setup
base_dir = 'data/'
fps_in = 33.33
exp = 4

# Resampling factor
upsample_factor = 2

# Read in data
subset = 1000
gcamp = np.genfromtxt(base_dir + str(exp) + '/gcamp_dff.csv', delimiter=',')
gcamp = gcamp[0:subset] # just look at subset of data for this example
cam_times = np.genfromtxt(base_dir + str(exp) + '/cam_times.csv', delimiter=',')
cam_times = cam_times[0:subset]
spike_times = np.genfromtxt(base_dir + str(exp) + '/spiketimes.csv', delimiter=',')

# Setup data dict for original data
data = {'calcium': gcamp,'fps': fps_in, 'cam_times': cam_times, 'spike_times': spike_times}

# Preprocess data: remove trends, upsample by upsample_factor, and normalize 
data = data_preprocessing.preprocess(data, factor = upsample_factor, low_perc = 1, up_perc = 80)

# Estimate decay rate based on known spike times 
decay_rate_estimates = estimate_decay_rates.estimate_gam(data)

print('Estimated AR1 decay rate \t ' + str(decay_rate_estimates['gam_hat']))

### Estimating spikes

Spikes are estimated via L0 penalization with the [FastLZeroSpikeInference](https://jewellsean.github.io/fast-spike-deconvolution/index.html) package. To estimate spikes we must provide the gcamp trace, decay rate, and a tuning parameter. Instead of specifying this parameter directly we provide a target average estimated firing rate. 

In [ ]:
# Estimate spikes based on estimated decay rate and a target average firing rate 
target_firing_rate = 5.0
out = estimate_spikes.estimate_spikes_by_firing_rate(data, decay_rate_estimates['gam_hat'], target_firing_rate)
# See out['fit'] for estimated quantities.  

print('------ Spike estimation complete -----')
print('Target firing rate \t' + str(target_firing_rate))
print('Estimated firing rate \t' + str(out['fit']['spikes'].shape[0] / (data['calcium'].shape[0] / data['fps'])))

### Visualize fit

In [ ]:
plt.figure()
plt.plot(data['cam_times'], data['calcium'],color='k', linestyle='--')
plt.plot(data['cam_times'], out['fit']['estimated_calcium'],color='#fdae61', lw = 0.8)
for st in out['fit']['spikes']:
    spike_time = data['cam_times'][st]
    plt.plot([spike_time, spike_time], [-.5, -.2], '#2b83ba', lw = 0.5)
    plt.ylim([-.8, 2.])


plt.xlabel('Time (s)')
plt.tight_layout()
plt.show()

## Example 2: An in-vivo data setting

Without ephys data, we do not estimate the decay rate. We just
- Load data and preprocess
- Estimate spikes based on (input) decay rate and target average firing rate

In [ ]:
fps_in = 30.0
decay_rate = 0.985207421875
target_firing_rate = 7

data = {'calcium': gcamp,'fps': fps_in}
data = data_preprocessing.preprocess(data, factor = 2, low_perc = 1, up_perc = 80)
out = estimate_spikes.estimate_spikes_by_firing_rate(data, decay_rate, target_firing_rate)


print('------ Spike estimation complete -----')
print('Target firing rate \t' + str(target_firing_rate))
print('Estimated firing rate \t' + str(out['fit']['spikes'].shape[0] / (data['calcium'].shape[0] / data['fps'])))

### Visualize results

Plot processed gcamp (black), estimated calcium (orange), and estimated spikes (blue lines)

In [ ]:
plt.figure()
plt.plot(data['cam_times'], data['calcium'],color='k', linestyle='--')
plt.plot(data['cam_times'], out['fit']['estimated_calcium'],color='#fdae61', lw = 0.8)
for st in out['fit']['spikes']:
    spike_time = data['cam_times'][st]
    plt.plot([spike_time, spike_time], [-.5, -.2], '#2b83ba', lw = 0.5)
    plt.ylim([-.8, 2.])


plt.xlabel('Time (s)')
plt.tight_layout()
plt.show()

## More information about the L0 problem
More information about the L0 penalized optimization is available via help and the package [webpage](https://jewellsean.github.io/fast-spike-deconvolution/index.html). 

In [ ]:
from FastLZeroSpikeInference import fast
help(fast)